In [4]:
import configparser, os
config = configparser.ConfigParser()
config.read('./keys.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

#replace the open ai key and googld search api key with your own
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPAPI_API_KEY'] = config['SERPAPI']['SERPAPI_API_KEY']

Model def using tools in the langchain agents for png metadata read, write, and google search of character info

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType, AgentExecutor
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain import SerpAPIWrapper
from PIL import Image
import base64
import json

#endable chat memory for agent to remember info from previous
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear();
#NOTE: chatgpt is horrible in using memory, even wrose than open source models. It's caused by its silly attempt to stop people using memory to ask for its prompt.
#never trust chatgpt with memory state. try to finish all the operations in one chain / one agent run.

#read chara info from PNG
def png_read(path:str, field:str):
    #path is the path of the original png
    #field is the json field to read from the png
    from PIL import Image
    import base64
    import json
    image = Image.open(path)
    info = image.info
    decoded_info = base64.b64decode(info['chara'])
    parsed_info = json.loads(decoded_info)
    return parsed_info['data'][field]

#write chara info to PNG
def png_write(path: str, field: str, new_data: str, new_path: str):
    #path is the path of the original png
    #field is the json field to write in the new png
    #new_data is the data to write in the new png
    #new_path is the path of the new png
    from PIL import Image
    import base64
    import json
    from PIL.PngImagePlugin import PngInfo
    
    #read the json info from the original png
    image = Image.open(path)
    info = image.info
    decoded_info = base64.b64decode(info['chara'])
    parsed_info = json.loads(decoded_info)
    
    #update the json info from the png
    parsed_info['data'][field] = new_data
    new_json_data = json.dumps(parsed_info)
    new_png_data = base64.b64encode(new_json_data.encode())
    new_chara = new_png_data.decode()
    
    #write the new png with the updated metadata
    metadata = PngInfo()
    metadata.add_text("chara", new_chara)
    image.save(new_path,format="PNG",pnginfo=metadata)
    return("PNG saved at: "+new_path)


SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = load_tools(["python_repl"])
tools += [
    StructuredTool.from_function(
        func=png_read,
        name="PNG read",
        description="useful for when you need to read the json info from PNG for character cards"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to search for new character info"
    ),
    StructuredTool.from_function(
        name = "PNG write",
        func = png_write,
        description="useful for when you need to update the json info from the original png and save it to the new png"
    )
]

llm = ChatOpenAI(temperature=0, model_name = 'gpt-3.5-turbo-16k')

#use STRUCTURED_CHAT for multiple input tools
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    memory=memory, 
    agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"]
    }
)


In [9]:
#main task, rewrite the description in the character card
memory.clear()
agent.run("read the description field from the character json info from kitty.png. rewrite the description field with the description of Amiya from Arknight series. write a long description with lots of details, using imagnation if needed. update the json info with amiya's description, and name it amiya.png.")



> Entering new  chain...
Action:
```
{
  "action": "PNG read",
  "action_input": {
    "path": "kitty.png",
    "field": "description"
  }
}
```
Observation: Kitty is a ferocious street cat who has been reincarnated into the body of a beautiful catgirl after dying. She has pale skin, a petite frame, bright green eyes, and long black hair. Two fluffy cat ears adorn her head. Her tail is long and skinny.

Kitty does not fully understand how to operate her human body so she is clumsy. Despite this, Kitty adapts to her human hands and quickly learns fine motor skills. She will alternate between walking on two legs and on all fours. 

Kitty has the mind of a feral street cat. Her previous life has made her always stay alert for danger. She does not understand human society or language. Kitty will speak in mangled, broken sentences and communicate mostly through feline noises. 

Kitty is aggressive and skittish. She hates being touched and will violently lash out at anyone who gets too clo

'The description field in the character json info from "kitty.png" has been successfully updated with Amiya\'s description from the Arknight series. The updated json info has been saved as "amiya.png".'

In [10]:
#verify the main task
png_read("amiya.png","description")

'Amiya is a character from the Arknight series. She is a young girl with silver hair and red eyes. She wears a white coat with black accents and carries a staff. Amiya is the leader of Rhodes Island, an organization that fights against the infected creatures known as Reunion. She is known for her determination, bravery, and strong sense of justice. Amiya is a skilled caster and has the ability to manipulate arts, making her a valuable asset in battles against the Reunion. She is also caring and compassionate, always looking out for the well-being of her teammates. With her leadership and powerful abilities, Amiya is a key figure in the fight to restore order and protect the innocent in the world of Arknight.'